# ~ Beagle 5.4 Homozygous Refererence Imputation Accuracy Experiment ~
# Part 1

### - This small experiment is meant to give an indication of how well the imputation program Beagle 5.4 can impute homozygous reference regions by only using variant calls for heterozygous and homozygous variants as input

### |====================================================================================|

# General Steps:
## 1. Create a complete VCF format dataframe of chromosome 1, including all homozygous references extracted from reference assembly
##    - This will function as the golden reference that the imputed chromosome region will be compared against
## 2. Impute a region of chromosome 1, with the imputation only using heterozygous and homozygous alt variants as input
## 3. Calculate concordance between imputed chr1 region and actual chr1 region

### Libraries and Functions

In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import multiprocessing
import time
import re
import wget
import os 
import shutil
import zipfile
import json
import subprocess
import psutil
import sys
import itertools

In [4]:
# Code From 1_preprocess_CGI_v2.py

def read_CGI(cgi_path):

    tr0 = time.time()
    print(f"\tReading file...")

    col_line = None
    with open(cgi_path, 'r') as f:
        for line in f:
            if '>' in line or 'locus' in line:
                col_line = col_line
                break

    cols = line.strip('\n').rsplit('\t')
    col_types = {col:str for col in cols}

    chunks = pd.read_csv(cgi_path, sep='\t', names = cols, header = None, comment = '#', dtype=col_types, chunksize=100000)
    df = pd.concat(chunks)
    df = df.iloc[1:]

    tr1 = round(time.time()-tr0,2)
    print(f"\tFinished reading, took {tr1}s")

    return df

def get_header_build(file_path):
    
    with open(file_path,"r") as f:
        header_info = []
        while True:
            line = f.readline()
            if line.startswith("#"):
                line = line.replace('\t', " ")
                line = line.replace('\n', "")
                header_info.append(line[1:])
            else:
                break
                
        build = None
                
        for x in header_info:
            if 'GENOME_REFERENCE' in x:
                if '36' in x:
                    print('Build 36')
                    build = 36
                elif '37' in x:
                    print('Build 37')
                    build = 37
                elif '38' in x:
                    print("Build 38")
                    build = 38
                else:
                    build = 'Unknown Build'
                    print("Unknown Build")

        return build,

def get_col_name(df, names):
    #returns the column string that has a matching search term in the list names

    cols = df.columns
    try:
        col_name = [col for col in cols if col in names][0]
    except Exception as e:
        return None
    return col_name


### Custom Filter script -> Leaves in homozygous reference region calls ###

def filter_chr(df):
    var_types = df['varType'].unique().tolist()
    var_types.remove('snp')
    var_types.remove('ref')
    remove_types = var_types
    
    #df = df[df['reference'].str.contains('=') == False] #Genotype is same as reference
    df = df[df['varType'].isin(remove_types) == False] #Exclude variant types that are not snp or reference equivalent
    df = df[df['reference'].str.len() == 1] #remove genotypes spanning multiple positions in a single line
    df = df[df['alleleSeq'].str.len() == 1] #remove genotypes spanning multiple positions in a single line
    
    
    #Remove low quality calls
    accepted_calls = ['PASS','VQHIGH',np.nan]
    call_rating_col = get_col_name(df, ['varQuality','varFilter']) #call rating column has a different name in different CGI files
    if call_rating_col:
        df = df[df[call_rating_col].isin(accepted_calls) == True]
        
    
    #Remove Mitochondrial SNPs
    df = df[df['chromosome'].str.contains('M') == False]
    
    
#     #Remove variation loci with single occurence: all should have a partner allele for determination of homozygous / heterozygous for alt
    
#     df_variants = df[df['reference']!='=']
#     are_duplicates = df_variants.duplicated('>locus', keep=False)
#     df_variants = df_variants[are_duplicates]
    
#     df_ref = df[df['reference'] == '=']
    
#     df = pd.concat([df_variants,df_ref])
#     df['begin'] = df['begin'].astype(int)
#     df['>locus'] = df['>locus'].astype(int)
#     df.sort_values(by=['begin','>locus'], inplace = True)
    
#     #Remove loci with single occurence (all loci should have a partner, those with single occurence had partner removed in one of the previous steps)
#     are_duplicates = df.duplicated('>locus', keep=False)
#     df = df[are_duplicates]
    
    #If rsid missing from external reference OR multiple rsids are present, replace with "."
    df['xRef'] = df['xRef'].fillna(".")
    df.loc[df['xRef'].str.contains(';'), 'xRef'] = '.'
    
    #Keep only rsid
    df['xRef'] = df['xRef'].str.split(':').str[1]
    df['xRef'] = df['xRef'].fillna(".")
    
    
#     df.reset_index(drop=True, inplace=True)
    return df

def build_vcf_df(cgi_df):
    
    df = cgi_df
    
    #Rename columns
    df.rename(columns = {'xRef':'ID', 
                         'end':'POS', 
                         'reference':'REF', 
                         'alleleSeq':'ALT', 
                         'chromosome':'CHROM'}, inplace = True)
    
    #Create QUAL, FILTER, INFO, FORMAT and NA00001 col 
    df['QUAL'] = '.'
    df['FILTER'] = '.'
    df['INFO'] = '.'
    df['FORMAT'] = 'GT'
    #df['NA00001'] = None
    
    #Remove loci with single occurence (all loci should have a partner, those with single occurence had partner removed in one of the previous steps)
    are_duplicates = df.duplicated('>locus', keep=False)
    df = df[are_duplicates]

    #Populate NA00001 with heterozygous/homozygous for alt allele 
    df['orig_ref_alt'] = df['REF'] + df['ALT']
  
    df_alt_count = df.groupby('>locus')['orig_ref_alt'].agg(list).reset_index()

    df_alt_count['ref'] = df_alt_count['orig_ref_alt'].str.get(0)
    df_alt_count['alt'] = df_alt_count['orig_ref_alt'].str.get(1)
    df_alt_count['alt1'] = df_alt_count['alt'].str[1]
    df_alt_count['alt2'] = df_alt_count['ref'].str[1]
    df_alt_count['ref'] = df_alt_count['ref'].str[0]

    df_alt_count['NA00001'] = np.where(df_alt_count['alt1'] != df_alt_count['alt2'], '0/1', '1/1')
    df_alt_count['NA00001'] = np.where((df_alt_count['alt1'] == df_alt_count['alt2']) & (df_alt_count['alt1'] == df_alt_count['ref']), '0/0', df_alt_count['NA00001'])

    df_alt_count = df_alt_count[['>locus', 'NA00001']]

    df = df.merge(df_alt_count, on='>locus', how='left')

    df['2nd_sort_col'] = np.where(df['REF'] != df['ALT'], 2, 1)
    #df['begin'] = df['begin'].astype(int)
    df = df.sort_values(by=['>locus', '2nd_sort_col'], ascending = False) #Very important for duplicates filter in following step
    
    
    #Replace alt alleles that are actually reference with "."
    df['ALT'] = np.where(df['REF'] == '=', ".", df['ALT'])

    are_duplicates = df.duplicated('>locus', keep='first')
    inverted = ~are_duplicates
    df = df[inverted]
    
    #Remove non-relevant columns
    df = df[['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'NA00001']]
    df['CHROM'] = df['CHROM'].str[3:] #Remove 'chr' prefix
    
    #Fill missing rsids
    df['ID'] = df['ID'].fillna(".")
    df['POS'] = df['POS'].astype(int)
    df = df.sort_values(by=['POS'], ascending = True)
    df.reset_index(drop=True, inplace=True)
    
    return df

def save_file(df, input_file_path, output_path):
    #Save processed file as a 23andMe-like .txt file

    build = get_header_build(input_file_path)
    new_header = ['fileformat=VCFv4.2',
                  'source=custom vcf transform script',
                  'reference={}'.format(build), 
                  'FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype>"']
                                  
    columns = df.columns.tolist()

    with open(output_path, 'w') as f:
        #Write Header
        for item in new_header: 
            f.write('##' + item +'\n')

        #Write Columns
        f.write('#')
        for col in columns:
            if col!= 'NA00001':
                f.write(col+'\t')
            else:
                f.write(col)
                f.write('\n')

    df.to_csv(r'{}'.format(output_path), header=False, index=None, sep='\t', mode='a')

# Load in File

In [103]:
#Complete Genomics raw input file
input_f = '/Users/jerenolsen/Desktop/Genomics_Pipeline_Versions/Genomics_Pipeline/input_genomes/hu1AF4ED_CGI_RawData.tsv'

In [33]:
df = read_CGI(input_f)

	Reading file...
	Finished reading, took 29.59s


### Comparing count of homozygous reference calls to variant calls in file

In [8]:
hom_ref = df[df['reference']=='=']
hom_ref = hom_ref[hom_ref['alleleSeq'] == '=']

In [10]:
hom_ref['hom_ref_count'] = hom_ref['end'].astype(int) - hom_ref['begin'].astype(int)

In [11]:
hom_ref.head()

,>locus,ploidy,allele,chromosome,begin,end,varType,reference,alleleSeq,varScoreVAF,varScoreEAF,varFilter,hapLink,xRef,alleleFreq,alternativeCalls,hom_ref_count
3,3,2,all,chr1,11430,11452,ref,=,=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22
5,5,2,all,chr1,11481,11504,ref,=,=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23
7,7,2,all,chr1,11511,11523,ref,=,=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
9,9,2,all,chr1,11575,11583,ref,=,=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
11,11,2,all,chr1,11609,11648,ref,=,=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39


In [12]:
total_hom_ref = hom_ref['hom_ref_count'].sum()
print(total_hom_ref+orig_len)
total_hom_ref/orig_len

2789813668


141.36365310141306

# Step 1: Create Complete chromosome 1 (Golden Reference)

In [14]:
chr1 = pd.DataFrame()
chr1 = pd.concat([df, chr1])
chr1 = chr1[chr1['chromosome'] == 'chr1']
chr1 = filter_chr(chr1)

In [15]:
chr1

,>locus,ploidy,allele,chromosome,begin,end,varType,reference,alleleSeq,varScoreVAF,varScoreEAF,varFilter,hapLink,xRef,alleleFreq,alternativeCalls
3,3,2,all,chr1,11430,11452,ref,=,=,NaN,NaN,NaN,NaN,.,NaN,NaN
5,5,2,all,chr1,11481,11504,ref,=,=,NaN,NaN,NaN,NaN,.,NaN,NaN
7,7,2,all,chr1,11511,11523,ref,=,=,NaN,NaN,NaN,NaN,.,NaN,NaN
9,9,2,all,chr1,11575,11583,ref,=,=,NaN,NaN,NaN,NaN,.,NaN,NaN
11,11,2,all,chr1,11609,11648,ref,=,=,NaN,NaN,NaN,NaN,.,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1569431,1237918,2,all,chr1,249238327,249238343,ref,=,=,NaN,NaN,NaN,NaN,.,NaN,NaN
1569433,1237920,2,all,chr1,249238350,249238410,ref,=,=,NaN,NaN,NaN,NaN,.,NaN,NaN
1569435,1237922,2,all,chr1,249238436,249238468,ref,=,=,NaN,NaN,NaN,NaN,.,NaN,NaN
1569437,1237924,2,all,chr1,249238469,249238479,ref,=,=,NaN,NaN,NaN,NaN,.,NaN,NaN


In [18]:
chr1_hom_ref = hom_ref[hom_ref['chromosome'] == 'chr1']
chr1_hom_ref.head()

,>locus,ploidy,allele,chromosome,begin,end,varType,reference,alleleSeq,varScoreVAF,varScoreEAF,varFilter,hapLink,xRef,alleleFreq,alternativeCalls,hom_ref_count
3,3,2,all,chr1,11430,11452,ref,=,=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22
5,5,2,all,chr1,11481,11504,ref,=,=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23
7,7,2,all,chr1,11511,11523,ref,=,=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
9,9,2,all,chr1,11575,11583,ref,=,=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
11,11,2,all,chr1,11609,11648,ref,=,=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39


In [19]:
chr1_hom_ref.reset_index(drop=True, inplace=True)

In [20]:
chr1_vars = pd.DataFrame()
chr1_vars = pd.concat([chr1[chr1['reference'] !='='], chr1_vars])
chr1_vars.head(6)

,>locus,ploidy,allele,chromosome,begin,end,varType,reference,alleleSeq,varScoreVAF,varScoreEAF,varFilter,hapLink,xRef,alleleFreq,alternativeCalls
774,774,2,1,chr1,38231,38232,snp,A,G,105,105,PASS,NaN,.,;,NaN
775,774,2,2,chr1,38231,38232,snp,A,G,239,239,PASS,NaN,.,;,NaN
793,792,2,1,chr1,38906,38907,ref,C,C,64,64,PASS,Phased_0_0_0,.,NaN,NaN
794,792,2,2,chr1,38906,38907,snp,C,T,125,125,PASS,Phased_0_0_1,.,;;,NaN
854,852,2,1,chr1,41980,41981,snp,A,G,482,482,PASS,NaN,rs806721,NaN,NaN
855,852,2,2,chr1,41980,41981,snp,A,G,179,179,PASS,NaN,rs806721,NaN,NaN


In [21]:
print(len(chr1))
print(len(chr1_vars))
print(len(chr1_hom_ref)) #Not including the count of begin and end ranges

1148580
529839
618741


### The following cell iteratres through the chromosome 1 dataframe that only contains homozygous reference regions. For each homozygous reference call in the df, samtools is called to extract the corresponding region from the GRCh37 reference file and appends the reference positions to a new dataframe

In [27]:
sam_dir = '/Users/jerenolsen/Desktop/All_Tests/Samtools'
cols = chr1.columns
new_entries = {col:[] for col in cols}
LOCUS = int(df['>locus'].astype(int).max())
len_chr1 = len(chr1)

unique_ref = 0
for num, row in chr1_hom_ref.iterrows():
    #t0=time.time()
    beg = None
    end = None
    extact_script = None
    if row['reference'] == '=':
        beg = int(row['begin'])
        end = int(row['end'])
    else:
        continue
        
    extract_script = "samtools faidx {sam_dir}/Homo_sapiens.GRCh37.dna.primary_assembly.fa 1:{beg}-{end}".format(beg=beg, end=end, sam_dir = sam_dir).split(" ")

    process = subprocess.run(extract_script, capture_output=True, text=True)
    outs = process.stdout.split()[1:]
    outs = "".join(outs)
    
    genotypes = list(outs)
    genotypes = list("".join(genotypes))
    num_positions = end-beg
    
    unique_ref+=1
    
    for i in range(0,num_positions):
        LOCUS +=1
        for j in range(1,3):
            genotype = genotypes[i]
            
            new_entries['>locus'].append(LOCUS)
            new_entries['ploidy'].append(2)
            new_entries['allele'].append(j)
            new_entries['chromosome'].append('chr1')
            new_entries['begin'].append(beg+i)
            new_entries['end'].append(beg+i+1)
            new_entries['varType'].append('ref')
            new_entries['reference'].append(genotype)
            new_entries['alleleSeq'].append(genotype)
            new_entries['varScoreVAF'].append(np.nan)
            new_entries['varScoreEAF'].append(np.nan)
            new_entries['varFilter'].append(np.nan)
            new_entries['hapLink'].append(np.nan)
            new_entries['xRef'].append(np.nan)
            new_entries['alleleFreq'].append(np.nan)
            new_entries['alternativeCalls'].append(np.nan)

    if num%1000 ==0:
        print(f"{num}/{len_chr1}")
        
    if num == 50000:
        break
        
df_hom_ref_chr1 = pd.DataFrame.from_dict(new_entries)
    

9700.41015625
7296.06640625
0/1148580
1000/1148580
2000/1148580
3000/1148580
4000/1148580
5000/1148580
6000/1148580
7000/1148580
8000/1148580
9000/1148580
10000/1148580
11000/1148580
12000/1148580
13000/1148580
14000/1148580
15000/1148580
16000/1148580
17000/1148580
18000/1148580
19000/1148580
20000/1148580
21000/1148580
22000/1148580
23000/1148580
24000/1148580
25000/1148580
26000/1148580
27000/1148580
28000/1148580
29000/1148580
30000/1148580
31000/1148580
32000/1148580
33000/1148580
34000/1148580
35000/1148580
36000/1148580
37000/1148580
38000/1148580
39000/1148580
40000/1148580
41000/1148580
42000/1148580
43000/1148580
44000/1148580
45000/1148580
46000/1148580
47000/1148580
48000/1148580
49000/1148580
50000/1148580


In [32]:
len(df_hom_ref_chr1)

16686438

In [35]:
#Get all variant positions from orignal chr1 dataframe that go up to the max position extracted from reference file
# in df_hom_ref_chr1
max_begin = df_hom_ref_chr1['begin'].astype(int).max()
chr1_variants = chr1_vars[chr1_vars['begin'].astype(int) <max_begin]

In [36]:
chr1_variants

,>locus,ploidy,allele,chromosome,begin,end,varType,reference,alleleSeq,varScoreVAF,varScoreEAF,varFilter,hapLink,xRef,alleleFreq,alternativeCalls
774,774,2,1,chr1,38231,38232,snp,A,G,105,105,PASS,NaN,.,;,NaN
775,774,2,2,chr1,38231,38232,snp,A,G,239,239,PASS,NaN,.,;,NaN
793,792,2,1,chr1,38906,38907,ref,C,C,64,64,PASS,Phased_0_0_0,.,NaN,NaN
794,792,2,2,chr1,38906,38907,snp,C,T,125,125,PASS,Phased_0_0_1,.,;;,NaN
854,852,2,1,chr1,41980,41981,snp,A,G,482,482,PASS,NaN,rs806721,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114610,99839,2,2,chr1,9670833,9670834,snp,C,T,247,247,PASS,Phased_5_71_1,rs4926472,NaN,NaN
114636,99865,2,1,chr1,9672090,9672091,snp,G,A,361,361,PASS,NaN,rs4073293,NaN,NaN
114637,99865,2,2,chr1,9672090,9672091,snp,G,A,43,43,PASS,NaN,rs4073293,NaN,NaN
114767,99993,2,1,chr1,9675375,9675376,snp,A,G,34,34,PASS,Phased_5_71_1,rs4078500,NaN,NaN


In [38]:
len(chr1_variants)

21375

In [39]:
#Remove the reference ranges in orignal chr1 df 
chr1_variants = chr1_variant[chr1_variants['reference'] != '=']

In [40]:
chr1_variants.head(-20)

,>locus,ploidy,allele,chromosome,begin,end,varType,reference,alleleSeq,varScoreVAF,varScoreEAF,varFilter,hapLink,xRef,alleleFreq,alternativeCalls
774,774,2,1,chr1,38231,38232,snp,A,G,105,105,PASS,NaN,.,;,NaN
775,774,2,2,chr1,38231,38232,snp,A,G,239,239,PASS,NaN,.,;,NaN
793,792,2,1,chr1,38906,38907,ref,C,C,64,64,PASS,Phased_0_0_0,.,NaN,NaN
794,792,2,2,chr1,38906,38907,snp,C,T,125,125,PASS,Phased_0_0_1,.,;;,NaN
854,852,2,1,chr1,41980,41981,snp,A,G,482,482,PASS,NaN,rs806721,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114391,99633,2,1,chr1,9653834,9653835,snp,A,G,242,242,PASS,NaN,rs3923984,NaN,NaN
114392,99633,2,2,chr1,9653834,9653835,snp,A,G,25,25,PASS,NaN,rs3923984,NaN,NaN
114396,99637,2,1,chr1,9654196,9654197,snp,T,C,467,467,PASS,NaN,rs7544083,NaN,NaN
114397,99637,2,2,chr1,9654196,9654197,snp,T,C,54,54,PASS,NaN,rs7544083,NaN,NaN


In [ ]:
#Combine chr1 variant calls and reference calls back together for complete chromosome subrange

In [41]:
chr1_combined = pd.concat([chr1_variants, df_hom_ref_chr1 ])

In [42]:
chr1_combined

,>locus,ploidy,allele,chromosome,begin,end,varType,reference,alleleSeq,varScoreVAF,varScoreEAF,varFilter,hapLink,xRef,alleleFreq,alternativeCalls
774,774,2,1,chr1,38231,38232,snp,A,G,105,105,PASS,NaN,.,;,NaN
775,774,2,2,chr1,38231,38232,snp,A,G,239,239,PASS,NaN,.,;,NaN
793,792,2,1,chr1,38906,38907,ref,C,C,64,64,PASS,Phased_0_0_0,.,NaN,NaN
794,792,2,2,chr1,38906,38907,snp,C,T,125,125,PASS,Phased_0_0_1,.,;;,NaN
854,852,2,1,chr1,41980,41981,snp,A,G,482,482,PASS,NaN,rs806721,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16686433,23766500,2,2,chr1,9676811,9676812,ref,G,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16686434,23766501,2,1,chr1,9676812,9676813,ref,C,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16686435,23766501,2,2,chr1,9676812,9676813,ref,C,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16686436,23766502,2,1,chr1,9676813,9676814,ref,C,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
chr1_combined['begin'] = chr1_combined['begin'].astype(int)
chr1_combined.sort_values(by=['begin','>locus'], inplace = True)
chr1_combined

,>locus,ploidy,allele,chromosome,begin,end,varType,reference,alleleSeq,varScoreVAF,varScoreEAF,varFilter,hapLink,xRef,alleleFreq,alternativeCalls
0,15423284,2,1,chr1,11430,11431,ref,C,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15423284,2,2,chr1,11430,11431,ref,C,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15423285,2,1,chr1,11431,11432,ref,T,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15423285,2,2,chr1,11431,11432,ref,T,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15423286,2,1,chr1,11432,11433,ref,G,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16686433,23766500,2,2,chr1,9676811,9676812,ref,G,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16686434,23766501,2,1,chr1,9676812,9676813,ref,C,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16686435,23766501,2,2,chr1,9676812,9676813,ref,C,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16686436,23766502,2,1,chr1,9676813,9676814,ref,C,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
len(chr1_combined)

16707813

In [46]:
df = pd.DataFrame()
df = pd.concat([chr1_combined,df])
df.head()

,>locus,ploidy,allele,chromosome,begin,end,varType,reference,alleleSeq,varScoreVAF,varScoreEAF,varFilter,hapLink,xRef,alleleFreq,alternativeCalls
0,15423284,2,1,chr1,11430,11431,ref,C,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15423284,2,2,chr1,11430,11431,ref,C,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15423285,2,1,chr1,11431,11432,ref,T,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15423285,2,2,chr1,11431,11432,ref,T,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15423286,2,1,chr1,11432,11433,ref,G,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Convert to VCF format

In [49]:
df_vcf_combined = build_vcf_df(df)

/var/folders/7s/_bh70s2n68zd14b4fsfcjcv00000gn/T/ipykernel_903/55039304.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['orig_ref_alt'] = df['REF'] + df['ALT']


           >locus ploidy allele CHROM    begin      POS varType REF ALT  \
1387379  16116427      2      1  chr1  1212635  1212636     ref   G   G   
1387380  16116427      2      2  chr1  1212635  1212636     ref   G   G   
1387377  16116426      2      1  chr1  1212634  1212635     ref   G   G   
1387378  16116426      2      2  chr1  1212634  1212635     ref   G   G   
1387375  16116425      2      1  chr1  1212633  1212634     ref   T   T   
1387376  16116425      2      2  chr1  1212633  1212634     ref   T   T   
1387373  16116424      2      1  chr1  1212632  1212633     ref   C   C   
1387374  16116424      2      2  chr1  1212632  1212633     ref   C   C   
1387371  16116423      2      1  chr1  1212631  1212632     ref   G   G   
1387372  16116423      2      2  chr1  1212631  1212632     ref   G   G   
1387369  16116422      2      1  chr1  1212630  1212631     ref   A   A   
1387370  16116422      2      2  chr1  1212630  1212631     ref   A   A   
1387367  16116421      2 

In [50]:
df_vcf_combined.head()

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,NA00001
0,1,11431,.,C,C,.,.,.,GT,0/0
1,1,11432,.,T,T,.,.,.,GT,0/0
2,1,11433,.,G,G,.,.,.,GT,0/0
3,1,11434,.,C,C,.,.,.,GT,0/0
4,1,11435,.,C,C,.,.,.,GT,0/0
5,1,11436,.,G,G,.,.,.,GT,0/0
6,1,11437,.,G,G,.,.,.,GT,0/0
7,1,11438,.,G,G,.,.,.,GT,0/0
8,1,11439,.,C,C,.,.,.,GT,0/0
9,1,11440,.,C,C,.,.,.,GT,0/0


# Save Golden Reference 

In [52]:
input_f = '/Users/jerenolsen/Desktop/Genomics_Pipeline/input_genomes/hu1AF4ED_CGI_RawData.tsv'
output_f = '/Users/jerenolsen/desktop/chr1_subsection_experiment_golden_reference.vcf'

In [53]:
save_file(df_vcf_combined,input_f, output_f)

Build 37


# Save Chromosome 1 subsection variants that will be input for the imputation program

In [54]:
df_vcf_vars = pd.DataFrame()
df_vcf_vars = pd.concat([chr1_variants, df_vcf_vars])
df_vcf_vars = build_vcf_df(df_vcf_vars)

/var/folders/7s/_bh70s2n68zd14b4fsfcjcv00000gn/T/ipykernel_903/55039304.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['orig_ref_alt'] = df['REF'] + df['ALT']


Empty DataFrame
Columns: [>locus, ploidy, allele, CHROM, begin, POS, varType, REF, ALT, varScoreVAF, varScoreEAF, varFilter, hapLink, ID, alleleFreq, alternativeCalls, QUAL, FILTER, INFO, FORMAT, orig_ref_alt, NA00001, 2nd_sort_col]
Index: []

[0 rows x 23 columns]


In [55]:
len(df_vcf_vars)

10182

In [56]:
df_vcf_vars_test = df_vcf_vars[df_vcf_vars['ALT']!= df_vcf_vars['REF']]

In [57]:
len(df_vcf_vars_test)

10181

In [59]:
df_vcf_vars

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,NA00001
0,1,38232,.,A,G,.,.,.,GT,1/1
1,1,38907,.,C,T,.,.,.,GT,0/1
2,1,41981,rs806721,A,G,.,.,.,GT,1/1
3,1,47108,rs2531241,G,C,.,.,.,GT,0/1
4,1,47292,rs2691275,T,G,.,.,.,GT,0/1
...,...,...,...,...,...,...,...,...,...,...
10177,1,9664650,rs138697503,T,A,.,.,.,GT,0/1
10178,1,9666364,rs7364899,A,G,.,.,.,GT,1/1
10179,1,9670483,rs4244632,T,A,.,.,.,GT,1/1
10180,1,9672091,rs4073293,G,A,.,.,.,GT,1/1


In [60]:
output_f = '/Users/jerenolsen/desktop/chr1_subsection_experiment_variants_tbi.vcf'

In [61]:
save_file(df_vcf_vars,input_f, output_f)
#df_vcf_vars.to_csv('/Users/jerenolsen/desktop/chr1_subsection_experiment_variants_tbi.vcf', sep = '\t')

Build 37


### ==============================================================
### ===| Perform Imputation of region, experiment continues in notebook 2 | ====
### ==============================================================